In [ ]:
# Data and calculation libraries
import numpy as np
import pandas as pd
import scipy.stats as st

In [ ]:
import mqr
from mqr.plot import Figure

In [ ]:
from IPython.display import display

---
# Toy examples

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
data = np.vstack([
    np.tile([1, 3], 10),
    np.tile([0, 0, 2, 2], 5)]).T
levels = pd.DataFrame(data, columns=['X', 'Y'])

---
# Linear

In [ ]:
# Make some data from a linear model, then add some noise to simulate measurement, unmodelled factors, etc.
np.random.seed(849)

linear = levels.copy()
f_linear = lambda x, y: x + 2*y
linear['Z'] = f_linear(linear['X'], linear['Y']) + st.norm(0, 0.5).rvs(len(levels))

In [ ]:
# Define and fit the model
result = smf.ols('Z ~ X + Y - 1', linear).fit()

In [ ]:
mqr.nbtools.vstack(
    mqr.anova.adequacy(result),
    sm.stats.anova_lm(result),
    mqr.anova.coeffs(result))

In [ ]:
Xs = np.linspace(0.5, 3.5)
Ys = np.linspace(-0.5, 2.5)
X, Y = np.meshgrid(Xs, Ys)
Z = f_linear(X, Y)

zmin, zmax = np.min(Z)-0.5, np.max(Z)+0.5

subplot_kw = {'projection': '3d'}
with Figure(5, 5, subplot_kw=subplot_kw) as (fig, ax):
    ax.plot_surface(X, Y, Z, edgecolor='C0', lw=0.5, rstride=4, cstride=4, alpha=0.0, color='C0')
    ax.plot(xs=linear['X'], ys=linear['Y'], zs=linear['Z'], linewidth=0, marker='.', color='C1')
    
    ax.contour(X, Y, Z, offset=np.min(Z)-0.5)
    ax.plot(1, 0, zs=zmin, color='k', marker='o')
    ax.plot(3, 0, zs=zmin, color='k', marker='o')
    ax.plot(1, 2, zs=zmin, color='k', marker='o')
    ax.plot(3, 2, zs=zmin, color='k', marker='o')
    
    ax.plot(1, 0, zs=f_linear(1, 0), color='C0', marker='o')
    ax.plot(3, 0, zs=f_linear(3, 0), color='C0', marker='o')
    ax.plot(1, 2, zs=f_linear(1, 2), color='C0', marker='o')
    ax.plot(3, 2, zs=f_linear(3, 2), color='C0', marker='o')
    
    ax.view_init(20, 240, 0)
    ax.set(
        xlim=(0.5, 3.5),
        ylim=(-0.5, 2.5),
        zlim=(zmin, zmax),
        xlabel='X',
        ylabel='Y',
        zlabel='Z')

In [ ]:
with Figure(6, 4, 2, 2) as (fig, ax):
    mqr.plot.regression.residuals(result, axs=ax)

---
# Linear with Interaction

In [ ]:
# Construct data with a "destructive" interaction, and fake some noisey measurements
np.random.seed(849)

interact = levels.copy()
f_interact = lambda x, y: x + 2 * y - x * y
interact['Z'] = f_interact(interact['X'], interact['Y']) + st.norm(0, 0.5).rvs(len(levels))

In [ ]:
Xs = np.linspace(0.5, 3.5)
Ys = np.linspace(-0.5, 2.5)
X, Y = np.meshgrid(Xs, Ys)
Z = f_interact(X, Y)

zmin, zmax = np.min(Z)-0.5, np.max(Z)+0.5

subplot_kw = {'projection': '3d'}
with Figure(5, 5, subplot_kw=subplot_kw) as (fig, ax):
    ax.plot_surface(X, Y, Z, edgecolor='C0', lw=0.5, rstride=4, cstride=4, alpha=0.0, color='C0')
    ax.plot(xs=interact['X'], ys=interact['Y'], zs=interact['Z'], linewidth=0, marker='.', color='C1')
    
    ax.contour(X, Y, Z, offset=np.min(Z)-0.5)
    ax.plot(1, 0, zs=zmin, color='k', marker='o')
    ax.plot(3, 0, zs=zmin, color='k', marker='o')
    ax.plot(1, 2, zs=zmin, color='k', marker='o')
    ax.plot(3, 2, zs=zmin, color='k', marker='o')
    
    ax.plot(1, 0, zs=f_interact(1, 0), color='C0', marker='o')
    ax.plot(3, 0, zs=f_interact(3, 0), color='C0', marker='o')
    ax.plot(1, 2, zs=f_interact(1, 2), color='C0', marker='o')
    ax.plot(3, 2, zs=f_interact(3, 2), color='C0', marker='o')
    
    ax.view_init(20, 240, 0)
    ax.set(
        xlim=(0.5, 3.5),
        ylim=(-0.5, 2.5),
        zlim=(zmin, zmax),
        xlabel='X',
        ylabel='Y',
        zlabel='Z')

In [ ]:
# Define the model and fit.
result = smf.ols('Z ~ X * Y - 1', interact).fit()

In [ ]:
display(mqr.anova.adequacy(result))
display(sm.stats.anova_lm(result))
display(mqr.anova.coeffs(result))

In [ ]:
with Figure(6, 4, 2, 2) as (fig, ax):
    mqr.plot.regression.residuals(result, axs=ax)

---
# Quadratic (and higher) curvature

This example uses a large amount of noise to show how effective linear regression is when the assumption about residuals holds.
That is: linear regression performs very well whenever the structure that the model does not capture (the residuals) is normally distributed.

Have a look at the large spread of the orange dots around the "true" surface response.
Also, notice that the $r^2_\textrm{adj}$ is fairly low because of this noise — around 0.8
(in practise, you should be suspicious that the model is poor or the measurements are too noisy).
Still, the estimate that regression produces is very good.

In [ ]:
np.random.seed(432)

curve = levels.copy()
# Add centrepoints to make the curvature detectable
curve = pd.concat(
    [curve,
     pd.DataFrame({'X': 2*np.ones(10),
                   'Y': np.ones(10)})],
    ignore_index=True)
f_curve = lambda x, y: -(x**2 - 3*x + 2) + 2*y
curve['Z'] = f_curve(curve['X'], curve['Y']) + st.norm(0, 1).rvs(len(curve))

In [ ]:
result = smf.ols('Z ~ X + I(X**2) + Y + 1', curve).fit()

In [ ]:
mqr.nbtools.vstack(
    mqr.anova.adequacy(result),
    sm.stats.anova_lm(result),
    mqr.anova.coeffs(result))

### Estimating a maximum from the fitted coefficients
The expression for the response surface was
\begin{align}
    z = ax^2 + bx - c + dy.
\end{align}
Setting the partial derivative wrt $x$ to zero gives
\begin{gather}
    x = -\frac{b}{2a}.
\end{gather}
And since we assumed the response to be linear in $y$, choose the highest feasible $y$ (since its coefficient is positive).
In this case, use $y=2$, the highest value for $y$ in the experiment.

In [ ]:
result.params

In [ ]:
d, b, a, c = result.params # careful here - look at result.params to see what order statsmodels/patsy used in the result
x_opt = -b / (2 * a)
y_opt = 2
z_opt = f_curve(x_opt, y_opt)
x_opt, z_opt, f_curve(1.5, 2) # optimal point from fit, then the optimal point from "truth"

In [ ]:
Xs = np.linspace(0.5, 3.5)
Ys = np.linspace(-0.5, 2.5)
X, Y = np.meshgrid(Xs, Ys)
Z = f_curve(X, Y)

zmin, zmax = np.min(Z)-0.5, np.max(Z)+0.5

subplot_kw = {'projection': '3d'}
with Figure(5, 5, subplot_kw=subplot_kw) as (fig, ax):
    ax.plot_surface(X, Y, Z, edgecolor='C0', lw=0.5, rstride=4, cstride=4, alpha=0.0, color='C0')
    ax.plot(xs=curve['X'], ys=curve['Y'], zs=curve['Z'], linewidth=0, marker='.', color='C1')
    
    ax.contour(X, Y, Z, offset=np.min(Z)-0.5)
    ax.plot(1, 0, zs=zmin, color='k', marker='o')
    ax.plot(3, 0, zs=zmin, color='k', marker='o')
    ax.plot(1, 2, zs=zmin, color='k', marker='o')
    ax.plot(3, 2, zs=zmin, color='k', marker='o')
    ax.plot(2, 1, zs=zmin, color='k', marker='o') # centre points
    
    ax.plot(1, 0, zs=f_curve(1, 0), color='C0', marker='o')
    ax.plot(3, 0, zs=f_curve(3, 0), color='C0', marker='o')
    ax.plot(1, 2, zs=f_curve(1, 2), color='C0', marker='o')
    ax.plot(3, 2, zs=f_curve(3, 2), color='C0', marker='o')
    ax.plot(2, 1, zs=f_curve(2, 1), color='C0', marker='o') # centre points

    ax.plot(x_opt, y_opt, zs=z_opt, color='red', marker='o') # fitted optimum
    xs_opt = np.linspace(x_opt-0.5, x_opt+0.5)
    ys_opt = np.linspace(y_opt-0.5, y_opt+0.5)
    ax.plot(xs_opt, np.repeat(y_opt, len(xs_opt)), f_curve(xs_opt, y_opt), color='red', linewidth=0.8)
    ax.plot(np.repeat(x_opt, len(ys_opt)), ys_opt, f_curve(x_opt, ys_opt), color='red', linewidth=0.8)
    
    ax.view_init(20, 240, 0)
    ax.set(
        xlim=(0.5, 3.5),
        ylim=(-0.5, 2.5),
        zlim=(zmin, zmax),
        xlabel='X',
        ylabel='Y',
        zlabel='Z')

In [ ]:
with Figure(6, 4, 2, 2) as (fig, ax):
    mqr.plot.regression.residuals(result, axs=ax)
    plot = mqr.nbtools.grab_figure(fig)

mqr.nbtools.hstack(
    plot,
    mqr.inference.dist.test_1sample(result.resid))

---
# Compressor Map
This examples shows an iterative process of eliminating insignificant coefficients.

The process performs the simple steps above, but remove coefficients after each iteration,
then repeats the process.

In [ ]:
df = pd.read_csv(mqr.sample_data('compressor-map.csv'))

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

---
### All terms up to cubic
The fit is pretty good, but the condition number is huge.
Drop higher order terms, starting with the highest pvalue (least likely to explain the endogenous var).

In [ ]:
# NB: the constant term is included by default, but typed out here for clarity
expr = f'''
    Capacity ~
    + C(Ts) + Td
    + C(Ts**2) + C(Ts):Td + I(Td**2)
    + C(Ts**3) + C(Ts**2):Td + C(Ts):I(Td**2) + I(Td**3)
    + 1
'''
model = smf.ols(expr, data=df)
res = model.fit()
table = res.summary2().tables[1]

In [ ]:
mqr.nbtools.vstack(
    np.linalg.cond(model.exog),
    mqr.anova.adequacy(res),
    table)

In [ ]:
import seaborn as sns

with Figure(6, 5) as (fig, ax):
    sns.heatmap(np.corrcoef(model.exog.T), xticklabels=model.exog_names, yticklabels=model.exog_names)

---
### Drop `Td**2` and `Td**2`

Fit improved a little, but still a huge condition number

In [ ]:
# NB: the constant term is included by default, but typed out here for clarity
expr = f'''
    Capacity ~
    + C(Ts) + Td
    + C(Ts**2) + C(Ts):Td
    + C(Ts**3) + C(Ts**2):Td + C(Ts):I(Td**2)
    + 1
'''
model = smf.ols(expr, data=df)
res = model.fit()
table = res.summary2().tables[1]

In [ ]:
mqr.nbtools.vstack(
    np.linalg.cond(model.exog),
    mqr.anova.adequacy(res),
    table)

---
### Drop squared interaction terms
r2adj hovering; still huge condition number.

In [ ]:
# NB: the constant term is included by default, but typed out here for clarity
expr = f'''
    Capacity ~
    + C(Ts) + Td
    + C(Ts**2) + C(Ts):Td
    + C(Ts**3)
    + 1
'''
model = smf.ols(expr, data=df)
res = model.fit()
table = res.summary2().tables[1]

In [ ]:
mqr.nbtools.vstack(
    np.linalg.cond(model.exog),
    mqr.anova.adequacy(res),
    table)

---
### Drop interaction term

In [ ]:
# NB: the constant term is included by default, but typed out here for clarity
expr = f'''
    Capacity ~
    + C(Ts) + Td
    + C(Ts**2)
    + C(Ts**3)
    + 1
'''
model = smf.ols(expr, data=df)
result = model.fit()
table = result.summary2().tables[1]

In [ ]:
mqr.nbtools.vstack(
    np.linalg.cond(model.exog),
    mqr.anova.adequacy(result),
    table)

---
### Plot residuals

In [ ]:
# Residuals are in result.resid
with Figure(7, 5, 2, 2) as (fig, ax):
    mqr.plot.regression.residuals(result, axs=ax)
    plot = mqr.nbtools.grab_figure(fig)

mqr.nbtools.hstack(
    plot,
    mqr.nbtools.vstack(
        mqr.inference.dist.test_1sample(result.resid, test='ad-norm'),
        mqr.inference.mean.test_1sample(result.resid)))